#### Upon initial examination of raw data file mmJC_020521.csv, there are two special cases which require handling:
- missing data 
- outliers (e.g., values>>100 for dO).
#### Additional notes:
- recordings in Liquid Handling (LH) are texts and could be stored in a separate table 
- For dO and pH, the extent of missing values fluctuate over time. Sometimes all chambers were missing chunks of data, while other times only some chambers have missing data.
- Data for O2, Air and N2 are sparse, but relatively consistent across chambers
- There were "UNKNOWN" records in the "Probe' column 

### Workflow
1. Reshape data from wide into long format and remove any missing values 
2. Identify outliers for each variable per chamber, remove outliers (i.e., treat as missing values)
3. Subset LH into a separate table because it is text data rather than numeric
4. Reshape cleaned data from long format back into wide format, downsampling time intervals (also help resolve inconsistent missing values across different times) for easy plotting

In [5]:
import pandas as pd
import numpy as np
import sys

In [2]:
# Load raw data
raw = pd.read_csv('../data/raw/mmJC_020521.csv', na_values = {"Probe":"UNKNOWN"})
raw.head()

,Hours,CS1-1-dO,CS1-2-dO,CS1-3-dO,CS1-4-dO,CS1-5-dO,CS1-6-dO,CS1-7-dO,CS1-8-dO,CS1-9-dO,...,CS2-4-Temperature,CS2-5-Temperature,CS2-6-Temperature,CS2-7-Temperature,CS2-8-Temperature,CS2-9-Temperature,CS2-10-Temperature,CS2-11-Temperature,CS2-12-Temperature,Probe
0,114.06,161.28,102.55,101.74,101.69,102.38,101.44,160.68,101.79,101.09,...,24.6,25.3,24.9,24.8,24.7,24.7,24.6,24.9,24.7,2/5/2021 11:09
1,114.07,102.47,102.55,101.74,101.69,100.90,101.44,101.93,101.79,101.09,...,25.5,25.9,25.3,24.2,25.1,25.0,25.0,25.2,24.8,2/5/2021 11:09
2,114.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.1,25.9,25.8,24.6,25.2,25.2,25.3,25.5,25.0,2/5/2021 11:09
3,114.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.5,24.9,24.9,25.1,24.9,25.0,25.1,25.2,24.7,2/5/2021 11:09
4,114.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.6,23.7,23.7,25.5,25.7,25.3,25.0,25.0,24.6,2/5/2021 11:09


In [3]:
list(raw.describe().loc['max',:]) # examine columns with possible outliers

[187.98,
 161.28,
 132.88,
 142.64,
 141.64,
 134.19,
 124.73,
 160.68,
 100000000.0,
 110.62,
 110.24,
 112.52,
 116.5,
 124.72,
 140.11,
 142.41,
 137.95,
 138.82,
 124.92,
 112.43,
 109.69,
 108.62,
 109.33,
 112.43,
 116.62,
 7.42,
 7.43,
 7.46,
 7.52,
 7.54,
 8.1,
 7.51,
 7.55,
 8.1,
 8.1,
 8.1,
 8.1,
 7.39,
 8.1,
 7.41,
 7.39,
 8.1,
 8.1,
 7.51,
 7.59,
 7.53,
 8.1,
 7.44,
 7.57,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 4.985,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 32.1,
 34.5,
 35.3,
 35.2,
 35.3,
 33.9,
 31.4,
 31.6,
 32.1,
 31.5,
 31.3,
 31.0,
 32.2,
 33.7,
 34.6,
 34.5,
 34.4,
 

In [4]:
# Generate timedelta for 'Hours' column, for resampling later.
raw['TimeDelta'] = pd.to_timedelta(raw['Hours'], unit='h')
raw['TimeDelta'].head()

0   4 days 18:03:36
1   4 days 18:04:12
2   4 days 18:04:48
3   4 days 18:06:00
4   4 days 18:07:12
Name: TimeDelta, dtype: timedelta64[ns]

In [5]:
# Reshape data and drop missing values
df_long = pd.melt(raw, id_vars = ['Probe','Hours','TimeDelta'],
                 var_name = 'id',
                 value_name = 'records').dropna(how='any')

In [6]:
print(df_long.shape)
df_long.head()

(109128, 5)


,Probe,Hours,TimeDelta,id,records
0,2/5/2021 11:09,114.06,4 days 18:03:36,CS1-1-dO,161.28
1,2/5/2021 11:09,114.07,4 days 18:04:12,CS1-1-dO,102.47
8,2/5/2021 11:09,114.18,4 days 18:10:48,CS1-1-dO,86.8
9,2/5/2021 11:09,114.20,4 days 18:12:00,CS1-1-dO,85.18
10,2/5/2021 11:09,114.22,4 days 18:13:12,CS1-1-dO,86.67


In [7]:
len(list(df_long['id'].unique()))

168

In [8]:
# Remove LH from dataframe and store in a separate table
LH = df_long[df_long.id.str.contains('LH')]
df_long = df_long[~df_long.id.str.contains('LH')]
df_long.shape

(108518, 5)

In [39]:
sum(df_long.id.str.contains('LH'))

0

In [9]:
# Remove outlier values (100000000)
# Future steps: we could write a function and adopt a criteria to systematically remove outliers
# Will need to consult the client about specific criteria, for example, 1.5*IQR
df_long = df_long.loc[df_long['records'] < 10000,:]
df_long.shape

(108514, 5)

In [10]:
# Split station name and variable name
df_long[['loc','chamber','variable']] = df_long.id.str.split('-',expand=True)
df_long['station_id'] = df_long['loc'] + '-' + df_long['chamber']

In [11]:
df_long.head()

,Probe,Hours,TimeDelta,id,records,loc,chamber,variable,station_id
0,2/5/2021 11:09,114.06,4 days 18:03:36,CS1-1-dO,161.28,CS1,1,dO,CS1-1
1,2/5/2021 11:09,114.07,4 days 18:04:12,CS1-1-dO,102.47,CS1,1,dO,CS1-1
8,2/5/2021 11:09,114.18,4 days 18:10:48,CS1-1-dO,86.8,CS1,1,dO,CS1-1
9,2/5/2021 11:09,114.20,4 days 18:12:00,CS1-1-dO,85.18,CS1,1,dO,CS1-1
10,2/5/2021 11:09,114.22,4 days 18:13:12,CS1-1-dO,86.67,CS1,1,dO,CS1-1


In [12]:
df_long = df_long.set_index('TimeDelta')
df_long['records'] = df_long['records'].apply(pd.to_numeric)
df_long.info()

<class 'pandas.core.frame.DataFrame'>
TimedeltaIndex: 108514 entries, 4 days 18:03:36 to 7 days 19:58:48
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Probe       108514 non-null  object 
 1   Hours       108514 non-null  float64
 2   id          108514 non-null  object 
 3   records     108514 non-null  float64
 4   loc         108514 non-null  object 
 5   chamber     108514 non-null  object 
 6   variable    108514 non-null  object 
 7   station_id  108514 non-null  object 
dtypes: float64(2), object(6)
memory usage: 7.5+ MB


In [13]:
list(df_long['variable'].unique())

['dO', 'pH', 'O2', 'Air', 'N2', 'Temperature']

In [14]:
# Loop through variables, subset data, reshape, downsample to 1H and drop rows with all NaN values
# Store results in excel sheets

variables = list(df_long['variable'].unique())

with pd.ExcelWriter('../data/processed/1H_mean.xlsx', datetime_format='hh:mm:ss.000') as writer:
    
    for var in variables:

        subset = df_long.loc[df_long['variable'] == var, ['station_id','records']]

        # reshape data into wide form
        subset_wide = subset.pivot_table(index = 'TimeDelta',values = 'records',columns='station_id')

        # Downsample data to 1 hour, taking averages, then drop rows in which all chambers had missing data
        subset_1H = subset_wide.resample('1H').mean().dropna(how='all')
        
        subset_1H = subset_1H.reset_index()
        subset_1H['TimeDelta'] = subset_1H['TimeDelta'].astype('str')
        
        subset_1H.to_excel(writer, sheet_name = var)
    
    


In [15]:
#### Exporter for database

# 1. fact table for variable records
cols = ['station_id','Hours','variable','records']
fact_records = df_long.reset_index()[cols] 

# 2. fact table for Liquid handling
# format LH table
LH[['loc','chamber','variable']] = LH.id.str.split('-',expand=True)
LH['station_id'] = LH['loc'] + '-' + LH['chamber']
LH.head()

fact_LH = LH.reset_index()[['station_id','Hours','records']]

# 3. dimension table for stations
dim_stations = pd.read_csv('../data/raw/mmJC_020521_meta.csv')
dim_stations.columns = ['station_name','treatment']

# 4. dimension table for variables 
dim_variable = pd.DataFrame(list(df_long['variable'].unique()))
dim_variable.columns = ['name']

# 5. dimension table for time
dim_time = df_long.reset_index()[['Hours','Probe']]

## Export to csv files
fact_records.to_csv('../data/processed/db/fact_records.csv')
fact_LH.to_csv('../data/processed/db/fact_LH.csv')
dim_stations.to_csv('../data/processed/db/dim_stations.csv')
dim_variable.to_csv('../data/processed/db/dim_variable.csv')
dim_time.to_csv('../data/processed/db/dim_time.csv')


